In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import string
from string import punctuation
import nltk
import re

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/DataSet/comments.csv')

In [ ]:
df = df1.drop(['Unnamed: 0','Video ID'], axis = 1)

In [ ]:
def numToWords(x):
    if x == 1.0:
        return "Neutral"
    elif x == 0.0:
        return "Negative"
    else:
        return "Positive"

In [ ]:
df['Sentiment'] = df['Sentiment'].apply(numToWords)

In [ ]:
data2 = pd.DataFrame()
data2['comment'] = df['Comment']
data2['sentiment'] = df['Sentiment']

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
stop_words = stopwords.words('english')
porter_stemmer = PorterStemmer()
lancaster_stemmer = LancasterStemmer()
snowball_stemer = SnowballStemmer(language="english")
lzr = WordNetLemmatizer()

In [ ]:
def text_processing(text):
    text = str(text);

    # convert text into lowercase
    text = text.lower()

    # remove new line characters in text
    text = re.sub(r'\n',' ', text)

    # remove punctuations from text
    text = re.sub('[%s]' % re.escape(punctuation), "", text)

    # remove references and hashtags from text
    text = re.sub("^a-zA-Z0-9$,.", "", text)

    # remove multiple spaces from text
    text = re.sub(r'\s+', ' ', text, flags=re.I)

    # remove special characters from text
    text = re.sub(r'\W', ' ', text)



    text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])

    text=' '.join([porter_stemmer.stem(word) for word in word_tokenize(text)])
    # text=' '.join([lancaster_stemmer.stem(word) for word in word_tokenize(text)])
    # text=' '.join([snowball_stemer.stem(word) for word in word_tokenize(text)])

    text=' '.join([lzr.lemmatize(word) for word in word_tokenize(text)])

    return text

In [ ]:
nltk.download('omw-1.4')
data_copy = data2.copy()
data_copy['comment'] = data_copy['comment'].apply(lambda tex: text_processing(str(tex)))

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
le = LabelEncoder()
data_copy['sentiment'] = le.fit_transform(data_copy['sentiment'])

In [ ]:
processed_data = {
    'comment':data_copy['comment'],
    'sentiment':data_copy['sentiment']
}

processed_data = pd.DataFrame(processed_data)
processed_data

,comment,sentiment
0,let forget appl pay 2014 requir brand new ipho...,1
1,nz 50 retail even contactless credit card mach...,0
2,forev acknowledg channel help lesson idea expl...,2
3,whenev go place take appl pay happen often dra...,0
4,appl pay conveni secur easi use use korean jap...,2
...,...,...
18404,realli like point engin toolbox think lot burn...,2
18405,start explor field realli good remind get earl...,2
18406,excelent video con una pregunta filosófica pro...,1
18407,hey daniel discov channel coupl day ago im lea...,2


In [ ]:

train_data = processed_data[:14000]
test_data = processed_data[14001:]

In [ ]:
!pip install datasets

In [ ]:
import datasets
from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def tokenize_function(batch):
    return tokenizer(batch, padding=True, truncation=True)

train_data['input_ids'] = train_data['comment'].apply(lambda x: tokenize_function(x)['input_ids'])
train_data['attention_mask'] = train_data['comment'].apply(lambda x: tokenize_function(x)['attention_mask'])

test_data['input_ids'] = test_data['comment'].apply(lambda x: tokenize_function(x)['input_ids'])
test_data['attention_mask'] = test_data['comment'].apply(lambda x: tokenize_function(x)['attention_mask'])
id2label = {
    "0": "negative",
    "1": "neutral",
    "2": "positive",
}
label2id = {
    "negative": 0,
    "neutral": 1,
    "positive": 2,
}

train_data['labels'] = train_data['sentiment']
test_data['labels'] = test_data['sentiment']

train_dataset = datasets.Dataset.from_pandas(train_data[['input_ids', 'attention_mask', 'labels']])
test_dataset = datasets.Dataset.from_pandas(test_data[['input_ids', 'attention_mask', 'labels']])



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

<ipython-input-16-dc5c1cca4923>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['input_ids'] = train_data['comment'].apply(lambda x: tokenize_function(x)['input_ids'])
<ipython-input-16-dc5c1cca4923>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['attention_mask'] = train_data['comment'].apply(lambda x: tokenize_function(x)['attention_mask'])
<ipython-input-16-dc5c1cca4923>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 3

model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels, id2label=id2label, label2id=label2id).to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 32
logging_steps = len(train_dataset) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=20,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    disable_tqdm=False,
    logging_steps=logging_steps,
    push_to_hub=True,
    log_level="error",
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.384300,0.526963,0.801044,0.794713
2,0.298100,0.507620,0.800590,0.798222
3,0.234700,0.675749,0.792196,0.778231
4,0.161600,0.687779,0.808303,0.800580
5,0.115300,0.857332,0.796506,0.799743
6,0.088300,0.979479,0.793330,0.791532
7,0.070500,1.157994,0.796733,0.785185
8,0.050800,1.223953,0.801270,0.802262
9,0.044900,1.300191,0.800590,0.799047
10,0.033100,1.314993,0.801724,0.800311


TrainOutput(global_step=8760, training_loss=0.08164704657631786, metrics={'train_runtime': 4164.34, 'train_samples_per_second': 67.238, 'train_steps_per_second': 2.104, 'total_flos': 1.1431660173487488e+16, 'train_loss': 0.08164704657631786, 'epoch': 20.0})

In [ ]:
model.save_pretrained(model_name)
tokenizer.save_pretrained(model_name)

('distilbert-base-uncased-finetuned-emotion/tokenizer_config.json',
 'distilbert-base-uncased-finetuned-emotion/special_tokens_map.json',
 'distilbert-base-uncased-finetuned-emotion/vocab.txt',
 'distilbert-base-uncased-finetuned-emotion/added_tokens.json',
 'distilbert-base-uncased-finetuned-emotion/tokenizer.json')

In [ ]:
model.save_pretrained('/content/drive/MyDrive/DataSet/trial3.h5')
tokenizer.save_pretrained('/content/drive/MyDrive/DataSet/trial3.h5')

('/content/drive/MyDrive/DataSet/trial3.h5/tokenizer_config.json',
 '/content/drive/MyDrive/DataSet/trial3.h5/special_tokens_map.json',
 '/content/drive/MyDrive/DataSet/trial3.h5/vocab.txt',
 '/content/drive/MyDrive/DataSet/trial3.h5/added_tokens.json',
 '/content/drive/MyDrive/DataSet/trial3.h5/tokenizer.json')

In [ ]:
preds_output = trainer.predict(test_dataset)
preds_output.metrics

{'test_loss': 1.579765796661377,
 'test_accuracy': 0.8071687840290381,
 'test_f1': 0.8036762654548413,
 'test_runtime': 22.171,
 'test_samples_per_second': 198.819,
 'test_steps_per_second': 6.224}

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_path = '/content/drive/MyDrive/DataSet/trial3.h5'
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
new_comments = pd.read_csv('sampleTest.csv')

def tokenize_comments(text):
    text = str(text)
    return tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')

tokenized_inputs = new_comments['text'].apply(tokenize_comments)

input_ids = torch.cat([x['input_ids'] for x in tokenized_inputs])
attention_mask = torch.cat([x['attention_mask'] for x in tokenized_inputs])

In [ ]:
input_ids = input_ids.to(model.device)
attention_mask = attention_mask.to(model.device)

with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    predictions = torch.argmax(outputs.logits, dim=-1)

predictions = predictions.cpu().numpy()

In [ ]:
id2label = {0: "negative", 1: "neutral", 2: "positive"}
new_comments['sentiment'] = [id2label[pred] for pred in predictions]

new_comments

,text,sentiment
0,Can't wait to see this one in everyone's recom...,positive
1,LOL very nice. I wasn't sure if someone had th...,positive
2,"Lorem ipsum dolor sit amet, consectetur adipis...",neutral
3,"You are the hero we need, but don't deserve. T...",positive
4,I know this sounds stupid but this is perfect ...,positive
...,...,...
84,jk,neutral
85,templates for workflows. Like you may have a p...,neutral
86,What is this,neutral
87,Thanks!,positive
